# RAG로 AI 와인 소믈리에 함수화 하기

## 환경변수 로딩

In [10]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

## 필요한 모듈 로딩

In [11]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

## LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력 : 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체 사용하기

In [15]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert who analyzes food images.
        When a user provides an image of a dish,
        identify the commonly recognized name of the dish, and
        clearly and concisely describe its flavor, focusing on the cooking method, texture, aroma, and balance of taste.
        If there is any uncertainty, base your analysis on the most likely dish, avoid definitive claims, and maintain a professional, expert tone.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래의 이미지의 요리에 대한 요리명과 요리의 풍미를 설명해 주세요.
            출력형태 :
            요리명:
            요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.        
        ])
    ])
    
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser

    return chain

In [16]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://thumbnail.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/9d0d/fd3f0d77757f64b2eba0905dcdd85051932ec1ab5e6afc0c3246f403fabc.jpg"
}

res = r1.invoke(input_data)

In [18]:
print(res)

요리명: 스테이크 (또는 구운 소고기 스테이크)

요리의 풍미:
이 스테이크는 겉면이 먹음직스럽게 캐러멜화되어 바삭한 식감을 선사하며, 내부의 선홍빛 육즙 가득한 살코기는 매우 부드럽고 촉촉할 것으로 예상됩니다. 조리 과정에서 발생한 고소하고 진한 육향이 코를 자극하며, 통후추의 은은한 매콤함과 로즈마리의 상쾌한 허브 향이 더해져 풍부한 아로마를 이룹니다. 굵은 소금 결정은 고기 본연의 풍미를 극대화하며 간의 균형을 잡아주어, savory하면서도 깊은 감칠맛이 입안 가득 조화롭게 어우러지는 풍미를 느낄 수 있습니다.
